In [43]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import joblib
df = pd.read_csv('houses_train.csv')
df.drop(['region', 'url','Unnamed: 0'], axis=1, inplace=True)
if 'price' in df.columns:
    y=df.pop('price')

In [38]:
x_train,x_test,y_train,y_test=train_test_split(df,y,train_size=0.9)
uniq=[]
for i in x_train.select_dtypes('object'):
    uniq.append(np.unique(x_train[i]))
    
ohec=OneHotEncoder(handle_unknown='ignore',sparse_output=False).fit(x_train.select_dtypes('object'))
x_train_num=PolynomialFeatures(degree=2,include_bias=False).fit_transform(x_train.select_dtypes('number').values)
x_train_obj=ohec.transform(x_train.select_dtypes('object'))
x_train1=np.concatenate([x_train_num,x_train_obj],axis=1)

model=Ridge(alpha=1)
model.fit(x_train1,y_train)

Ridge(alpha=1)

In [39]:
class house_price_predictor():
    def __init__(self,model,ohec,uniq,x_train):
        import pandas as pd
        import numpy as np
        from sklearn.linear_model import Ridge
        from sklearn.metrics import r2_score
        from sklearn.preprocessing import OneHotEncoder
        from sklearn.preprocessing import PolynomialFeatures
        from sklearn.model_selection import KFold
        self.model=model
        self.ohec=ohec
        self.uniq=uniq
        self.x_train=x_train.select_dtypes('object')
    def prep_for_predict(self,data):
        x_test_obj=data.select_dtypes('object')
        for i in range(len(x_test_obj)):
            a=[]
            for j in range(len(x_test_obj.iloc[i])):
                if x_test_obj.iloc[i,j] not in uniq[j]:
                    a.append(j)
            if a:
                x_test_obj.iloc[i]=self.handle_unseen(a,x_test_obj.iloc[i])
        data=pd.concat([data.select_dtypes('number'),x_test_obj],axis=1)
        return data
    def handle_unseen(self,l,x):
        print(x.values)
        arr=[]
        for i in range(len(self.x_train)):
            a=0
            for j in range(len(self.x_train.iloc[i])):
                if x[j] != self.x_train.iloc[i][j]:
                    a+=1
            arr.append(a)
        arr=np.array(arr)
        idmin=np.argmin(arr)
        for i in l:
            x[i]=self.x_train.iloc[idmin,i]
        print(x.values)
        return x
        
        
    def score(self,x,y,handle_unseen='do nothing'):
        assert type(x) == pd.core.frame.DataFrame, "data should be pandas dataframe type"
        if handle_unseen=='handle':
            x=self.prep_for_predict(x)
        x_test_num=PolynomialFeatures(degree=2,include_bias=False).fit_transform(x.select_dtypes('number').values)
        x_test_cat=self.ohec.transform(x.select_dtypes('object'))
        x_test=np.concatenate([x_test_num,x_test_cat],axis=1)
        y_pred=self.model.predict(x_test)
        return r2_score(y,y_pred)

final=house_price_predictor(model,ohec,uniq,x_train)

In [40]:
final.score(x_test,y_test)

0.8109736371673452

In [41]:
final.score(x_test,y_test,handle_unseen='handle')

['good' 'Nor Norq' 'Muradyan St' 'panel']
['good' 'Nor Norq' 'Mar St' 'panel']
['newly repaired' 'Erebuni' 'Lisinyan st.' 'stone']
['newly repaired' 'Erebuni' 'Artsakh ave' 'stone']
['newly repaired' 'Center' 'Koryun alley' 'stone']
['newly repaired' 'Center' 'Tigran Mets Ave' 'stone']
['newly repaired' 'Shengavit' 'Aragats St' 'panel']
['newly repaired' 'Shengavit' 'Tadevosyan St' 'panel']
['newly repaired' 'Erebuni' 'Lisinyan st.' 'stone']
['newly repaired' 'Erebuni' 'Artsakh ave' 'stone']
['good' 'Center' 'Tigran Mets Ave 2 alley' 'stone']
['good' 'Center' 'Kilikia district' 'stone']
['newly repaired' 'Nor Norq' 'Vaxtangov St' 'panel']
['newly repaired' 'Nor Norq' 'Vilnyus St' 'panel']
['good' 'Nor Norq' 'Mar alley' 'stone']
['good' 'Nor Norq' 'Gyurjyan St' 'stone']
['zero condition' 'Avan' 'Avan 4 St' 'monolit']
['zero condition' 'Avan' 'Tsarav Aghbyur St' 'monolit']
['newly repaired' 'Qanaqer-Zeytun' 'Kanaker 1 St' 'stone']
['newly repaired' 'Qanaqer-Zeytun' 'Davit Anhaxt St' 'sto

0.8077106941786556